<a href="https://colab.research.google.com/github/sirius70/NLP_HW2/blob/main/reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Sentiment Labels

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents.  As we discussed in class, many classification models represent a document as a sparse vector of of features and their weights. Weights could be boolean, counts, tf-idf values, or other functions of the document. In a linear model, we then combine the document vectors with the candidate output classes and learn weights for each of the feature-class pairs. This last step is implemented for you.

For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

There are many helpful resources online for getting up to speed with vector representations of documents. One example is the first two chapters of Delip Rao and Brian McMahan, _Natural Language Processing with PyTorch_, O'Reilly, 2019.  You should be able to <a href="https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/">read it online</a> via the Northeastern Library's subscription using a <tt>northeastern.edu</tt> email address.

In [1]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
import re
from scipy import sparse

In [2]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [3]:
def tokenize(text):
    """
    Tokenizes text into words that contain at least one alphabetic character.
    Splits on underscores and other non-word characters.
    """
    # Replace underscores with spaces so they're split
    text = text.replace("_", " ")

    # Match tokens with at least one alphabetic character
    return re.findall(r'\b\w*[a-zA-Z]\w*\b', text.lower())

In [4]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
    # Extract texts and classes
    texts = [entry["text"] for entry in corpus]
    classes = [entry["class"] for entry in corpus]

    # Step 1: Build vocabulary from all documents
    vocab_set = set()
    tokenized_docs = []

    for text in texts:
        tokens = tokenize(text)
        vocab_set.update(tokens)
        tokenized_docs.append(tokens)

    # Step 2: Create sorted vocabulary and word-to-index mapping
    vocab = sorted(list(vocab_set))
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}

    # Step 3: Build sparse matrix
    row_indices = []
    col_indices = []
    data = []

    for doc_idx, tokens in enumerate(tokenized_docs):
        word_counts = Counter(tokens)
        for word, count in word_counts.items():
            if word in word_to_idx:
                row_indices.append(doc_idx)
                col_indices.append(word_to_idx[word])
                data.append(count)

    X = sparse.csr_matrix((data, (row_indices, col_indices)),
                          shape=(len(texts), len(vocab)))

    return X, classes, vocab


In [6]:
# TODO: Implement createFancyFeatures andn describe in comments what the
# features are and why they might ben helpful.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.

def createFancyFeatures(corpus):
    """
    Creates a sparse feature matrix with 'fancy' features for sentiment classification.

    Features included:
    1. Unigrams: single words with at least one alphabetic character.
       - Useful because individual words often carry sentiment (e.g., "good", "bad").
    2. Bigrams: sequences of two consecutive words.
       - Useful because phrases can capture context and negation (e.g., "not good", "very bad"),
         which single words alone may miss.
    """
    texts = [doc["text"] for doc in corpus]
    classes = [doc["class"] for doc in corpus]

    tokenized_docs = []
    unigram_vocab = set()
    bigram_vocab = set()

    # Step 1: tokenize and collect unigrams and bigrams
    for text in texts:
        tokens = tokenize(text)
        tokenized_docs.append(tokens)
        unigram_vocab.update(tokens)
        bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens)-1)]
        bigram_vocab.update(bigrams)

    # Step 2: combine unigrams + bigrams into final vocab
    vocab = sorted(list(unigram_vocab)) + sorted(list(bigram_vocab))
    vocab_index = {tok: idx for idx, tok in enumerate(vocab)}

    # Step 3: build sparse feature matrix
    row, col, data = [], [], []

    for doc_id, tokens in enumerate(tokenized_docs):
        # Add unigram counts
        counts = Counter(tokens)
        for tok, cnt in counts.items():
            row.append(doc_id)
            col.append(vocab_index[tok])
            data.append(cnt)
        # Add bigram counts
        bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens)-1)]
        counts_bg = Counter(bigrams)
        for bg, cnt in counts_bg.items():
            row.append(doc_id)
            col.append(vocab_index[bg])
            data.append(cnt)

    X = sparse.csr_matrix((data, (row, col)), shape=(len(texts), len(vocab)))

    return X, classes, vocab


In [7]:
# given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  # create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))

  # determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  # determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [8]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [9]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.826500
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'sherri', 'enjoyable', 'perfectly', 'edge', 'masterpiece', 'excellent', 'gas', 'overall', 'using', 'fun', 'quite', 'follows', 'solid', 'command', 'holds', 'different', 'loved']
The most informative terms for neg are: ['waste', 'mess', 'ridiculous', 'lame', 'worst', 'headed', 'awful', 'cheap', 'unfortunately', 'write', 'boring', 'superior', 'tedious', 'bad', 'jesse', 'terrible', 'poor', 'looks', 'maybe', 'flat']
----------L2 Norm-----------
The model's average accuracy is 0.834000
The most informative terms for pos are: ['fun', 'back', 'great', 'quite', 'well', 'excellent', 'overall', 'seen', 'american', 'perfectly', 'memorable', 'yet', 'terrific', 'job', 'pulp', 'true', 'performances', 'very', 'solid', 'different']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'only', 'script', 'boring', 'awful',

Run the following to train and evaluate two models using extended features:

In [10]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.831500
The most informative terms for pos are: ['even if', 'flaws', 'memorable', 'terrific', 'overall', 'due to', 'the more', 'masterpiece', 'follows', 'seen', 'by a', 'her husband', 'gas', 'works', 'others', 'loved', 'great', 'filmmaking', 'as much', 'just about']
The most informative terms for neg are: ['ridiculous', 'waste', 'unfortunately', 'mess', 'worst', 'awful', 'cheap', 'boring', 'should have', 'lame', 'headed', 'poor', 'bad', 'write', 'maybe', 'designed', 'nowhere', 'terrible', 'superior', 'metro']
----------L2 Norm-----------
The model's average accuracy is 0.852000
The most informative terms for pos are: ['great', 'well', 'fun', 'seen', 'very', 'back', 'also', 'quite', 'many', 'people', 'yet', 'american', 'life', 'while', 'see', 'the two', 'job', 'excellent', 'most', 'perfectly']
The most informative terms for neg are: ['bad', 'only', 'unfortunately', 'worst', 'plot', 'nothing', 'any', 'boring', 'waste', 'script

**TODO**: Briefly comment on your results. You do _not_ need to beat the basic features model to get full credit.